In [ ]:
try:
  from datasets import load_dataset
except:
  !pip install datasets
  from datasets import load_dataset

ds = load_dataset("OpenAssistant/oasst1")
train = ds['train']
val = ds['validation']

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

(…)-00000-of-00001-b42a775f407cee45.parquet:   0%|          | 0.00/39.5M [00:00<?, ?B/s]

(…)-00000-of-00001-134b8fd0c89408b6.parquet:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/84437 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4401 [00:00<?, ? examples/s]

In [ ]:
# Фильтрация для топ-уровневых инструкций на русском языке
def filter_russian_top_level(example):
    return example['lang'] == 'ru' and example['parent_id'] is None

# Применение фильтрации
train_filtered = train.filter(filter_russian_top_level)
val_filtered = val.filter(filter_russian_top_level)

# Вывод размера отфильтрованных данных
print(f"Отфильтрованный train: {len(train_filtered)} записей")
print(f"Отфильтрованный val: {len(val_filtered)} записей")

# Пример первых записей
print(train_filtered[:5])  # Вывести первые 5 записей для проверки


Filter:   0%|          | 0/84437 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4401 [00:00<?, ? examples/s]

Отфильтрованный train: 721 записей
Отфильтрованный val: 39 записей
{'message_id': ['e480f611-0d31-433a-93d2-0e2bc675aa30', 'fa8c92d6-6daa-42a3-a2f5-e2e0eb610c41', 'ef69d986-aa2d-4c3e-a122-1910d9625c79', 'e321f8f6-01f8-4975-afe6-73db989f538e', '5083658c-e251-4259-b180-77afaf907d66'], 'parent_id': [None, None, None, None, None], 'user_id': ['05a999f8-8219-4584-a3f2-d4b4aec30027', '19241e3c-8272-4c3f-afe8-527c005d042c', '6b1efbfa-7c16-47bf-b90c-340ab2b1b7eb', '3c8cde8c-9466-494f-9c36-cbf3c3ac7ecd', '05a999f8-8219-4584-a3f2-d4b4aec30027'], 'created_date': ['2023-02-06T23:09:05.356836+00:00', '2023-02-07T14:36:48.863674+00:00', '2023-02-08T14:59:45.846381+00:00', '2023-02-08T14:19:18.286994+00:00', '2023-02-07T15:32:50.130297+00:00'], 'text': ['Напиши информацию о игре Hytale', 'Докончи поговорку, без труда не выловишь и', 'У меня угнали машину, что мне делать?', 'Какие произведения Булгакова связаны с Иерусалимом?', 'Плюсы и минусы языков программирования C++ и Rust. Что лучше?'], 'role': 

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Загрузка модели и токенизатора
model_name = "IlyaGusev/saiga_llama3_8b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Пример генерации ответа на инструкцию
def generate_response(instruction, tokenizer, model):
    inputs = tokenizer(instruction, return_tensors="pt", truncation=True, padding=True).to(device)
    output = model.generate(**inputs, max_new_tokens=200)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Функция для оценки ответов по критериям
def evaluate_response(response, tokenizer, model):
    evaluation_prompt = f"Оцени этот ответ по критериям сарказма: \"{response}\". Пожалуйста, дай оценку от 1 до 5."
    inputs = tokenizer(evaluation_prompt, return_tensors="pt").to(device)
    outputs = model.generate(inputs['input_ids'], max_length=10, num_return_sequences=1)
    score = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return score.strip()  # Возвращаем оценку без лишних пробелов

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# Собираем данные для DPO
data_for_dpo = []

# Пример генерации для нескольких инструкций
for instruction in train_filtered['text'][:5]:  # Замените на ваши данные
    response = generate_response(instruction, tokenizer, model)
    print(f"Инструкция: {instruction}")
    print(f"Ответ: {response}\n")
    score = evaluate_response(response, tokenizer, model)
    print(f'Оценка: {score}')

    # Добавляем данные в список для DPO
    data_for_dpo.append({
        "instruction": instruction,
        "response": response,
        "score": int(score)  # Приводим к целому числу
    })

In [ ]:
# Генерация ответов для базовой модели
base_responses = []
for instruction in val_filtered['text']:
    response = generate_response(instruction, tokenizer, model)  # Можно использовать ту же функцию, если модель загружена
    base_responses.append(response)

In [ ]:
# Функция для DPO
def dpo_training(data):
    model.train()
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

    # Проходим по всем парам в данных
    for i in range(len(data)):
        for j in range(len(data)):
            if i != j:  # Сравниваем только разные ответы
                inst_i = data[i]["instruction"]
                resp_i = data[i]["response"]
                score_i = data[i]["score"]
                inst_j = data[j]["instruction"]
                resp_j = data[j]["response"]
                score_j = data[j]["score"]

                # Сравнительная оптимизация
                if score_i > score_j:
                    prompt = f"Инструкция: {inst_i}\nОтвет: {resp_i}\nОценка: {score_i}\n" \
                             f"Инструкция: {inst_j}\nОтвет: {resp_j}\nОценка: {score_j}\n"
                    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(device)
                    outputs = model(**inputs, labels=inputs.input_ids)
                    loss = outputs.loss
                    loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()

# Обучаем модель на собранных данных
dpo_training(data_for_dpo)

print("Модель успешно дообучена с использованием DPO.")

In [ ]:
# Генерация ответов для валидационной выборки
val_responses = []
for instruction in val_filtered['text']:  # Замените на ваши данные
    response = generate_response(instruction, tokenizer, model)
    val_responses.append(response)


In [ ]:
val_filtered['base_response'] = base_responses
val_filtered['trained_response'] = val_responses

In [ ]:
# Оценка ответов для базовой и дообученной модели
val_results = []
eval_model_name = "sberbank-ai/rugpt3large_based_on_gpt2"
eval_tokenizer = AutoTokenizer.from_pretrained(model_name)
eval_model = AutoModelForCausalLM.from_pretrained(model_name)

for index, row in val_filtered.iterrows():
    instruction = row['text']
    base_response = row['base_response']
    trained_response = row['trained_response']

    # Оценка ответа от базовой модели
    base_score = evaluate_response(base_response, eval_tokenizer, eval_model)  # Используйте вашу LLM для оценки
    # Оценка ответа от обученной модели
    trained_score = evaluate_response(trained_response, eval_tokenizer, eval_model)

    val_results.append({
        "instruction": instruction,
        "base_response": base_response,
        "trained_response": trained_response,
        "base_score": base_score,
        "trained_score": trained_score
    })

# Создаем DataFrame с результатами
val_results_df = pd.DataFrame(val_results)


In [ ]:
# Выводим средние оценки
mean_base_scores = val_results_df['base_score'].mean()
mean_trained_scores = val_results_df['trained_score'].mean()

print("Средняя оценка для базовой модели:", mean_base_scores)
print("Средняя оценка для обученной модели:", mean_trained_scores)

# Визуализация сравнения
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.bar(['Base Model', 'Trained Model'], [mean_base_scores, mean_trained_scores], color=['blue', 'orange'])
plt.ylabel('Средняя оценка')
plt.title('Сравнение оценок базовой и обученной модели на валидационной выборке')
plt.show()

# Дополнительный анализ: Распределение оценок
plt.figure(figsize=(12, 6))
plt.hist(val_results_df['base_score'], bins=5, alpha=0.5, color='blue', label='Base Model')
plt.hist(val_results_df['trained_score'], bins=5, alpha=0.5, color='orange', label='Trained Model')
plt.title('Распределение оценок моделей на валидационной выборке')
plt.xlabel('Оценка')
plt.ylabel('Частота')
plt.legend()
plt.grid()
plt.show()
